In [1]:
import pickle as pkl 
import json

In [2]:
mistral_finetune_length_path = "/scratch/tathagato/fsdp_qlora_experiments_26_August_mistral/length/result.pkl"
mistral_finetune_topic_path = "/scratch/tathagato/fsdp_qlora_experiments_26_August_mistral/topic/result.pkl"
mistral_zero_shot_length_path = "/scratch/tathagato/fsdp_qlora_experiments_26_August_mistral/zero_shot_inference/result_length.pkl"
mistral_zero_shot_topic_path = "/scratch/tathagato/fsdp_qlora_experiments_26_August_mistral/zero_shot_inference/result_topic.pkl"

# Load the results
with open(mistral_finetune_length_path, "rb") as f:
    mistral_finetune_length = pkl.load(f)
with open(mistral_finetune_topic_path, "rb") as f:
    mistral_finetune_topic = pkl.load(f)
with open(mistral_zero_shot_length_path, "rb") as f:
    mistral_zero_shot_length = pkl.load(f)
with open(mistral_zero_shot_topic_path, "rb") as f:
    mistral_zero_shot_topic = pkl.load(f)


#print the number of keys in each dictionary
print("Number of keys in mistral_finetune_length: ", len(mistral_finetune_length.keys()))
print("Number of keys in mistral_finetune_topic: ", len(mistral_finetune_topic.keys()))
print("Number of keys in mistral_zero_shot_length: ", len(mistral_zero_shot_length.keys()))
print("Number of keys in mistral_zero_shot_topic: ", len(mistral_zero_shot_topic.keys()))



Number of keys in mistral_finetune_length:  554
Number of keys in mistral_finetune_topic:  272
Number of keys in mistral_zero_shot_length:  554
Number of keys in mistral_zero_shot_topic:  272


In [3]:
#see an example of the first key in first dictionary 
print("Example of first key in mistral_finetune_length: ", list(mistral_finetune_length.keys())[0])
for key in mistral_finetune_length[0].keys():
    print(f"{key} : \n\n {mistral_finetune_length[0][key]} \n ---------------------------------------------\n")
for key in mistral_finetune_topic[0].keys():
    print(f"{key} : \n\n {mistral_finetune_topic[0][key]} \n ---------------------------------------------\n")


Example of first key in mistral_finetune_length:  0
input : 

 Write a summary of the source text. The summary should be short in length. The length is defined in terms of number of words used in the summary. The source text is given below.  (CNN)Jackson Gordon is no ordinary 21-year-old. By day he is an industrial design student at Philadelphia University, but Gordon has another side to him -- a side altogether darker, tougher and more enigmatic. Hanging in his workshop Gordon has a full suit of armor plating, cape and cowl -- matte black and built to stop a knife. Gordon has an alter ego: the Dark Knight himself, Batman. You might expect his origin story to be cloaked in mystery, but speaking to CNN Gordon is quick to explain how the transformation took place. Gordon says his calling came five years ago when he began experimenting with cosplay. "Previously I'd been involved with costume making... I'd made a version of the Batsuit from Christopher Nolan's 'Dark Knight Trilogy' and I r

In [4]:
# get the control value by the output 
val_dataset_path = "/home2/tathagato/summarization/MACSUM/dataset/macdoc/val_dataset.json"
with open(val_dataset_path, "r") as f:
    val_dataset = json.load(f)
    

def cleanup_output_dict(output_dict, attribute = "length"):
    new_output_dict = {}
    for key in output_dict.keys():
        reference = output_dict[key]["reference"]
        new_output_dict[key] = {}
        done = False 
        for val_key in val_dataset.keys():
            val_reference = val_dataset[val_key]["reference"]
            if reference == val_reference:
                new_output_dict[key]["control_value"] = val_dataset[val_key]["control_attribute"][attribute]
                new_output_dict[key]["control_attribute"] = attribute
                new_output_dict[key]["output"] = output_dict[key]["reference"]
                new_output_dict[key]['predicted_summary'] = output_dict[key]["summary"]
                new_output_dict[key]['prompt'] = output_dict[key]["input"]
                new_output_dict[key]['generated_text'] = output_dict[key]["generated_text"]
                new_output_dict[key]['input'] = val_dataset[val_key]["source"]
                done = True
                break
        if not done:
            print("Error: Could not find the reference in the validation dataset")
    return new_output_dict
        

In [5]:
processed_zero_shot_length = cleanup_output_dict(mistral_zero_shot_length, "length")
processed_zero_shot_topic = cleanup_output_dict(mistral_zero_shot_topic, "topic")

#for key in mistral_finetune_length[0].keys():
#     print(f"{key} : \n\n {mistral_finetune_length[0][key]} \n ---------------------------------------------\n")
# for key in mistral_finetune_topic[0].keys():
#     print(f"{key} : \n\n {mistral_finetune_topic[0][key]} \n ---------------------------------------------\n")
for i in range(5):
    for key in processed_zero_shot_length[i].keys():
        print(f"{key} : \n\n {processed_zero_shot_length[i][key]} \n ---------------------------------------------")
print("---------------------------------------------\n\n")
for i in range(5):
    for key in processed_zero_shot_topic[i].keys():
        print(f"{key} : \n\n {processed_zero_shot_topic[i][key]} \n ---------------------------------------------")




control_value : 

 short 
 ---------------------------------------------
control_attribute : 

 length 
 ---------------------------------------------
output : 

 Jackson Gordon, a 21-year-old industrial design student at Philadelphia University built a Batsuit that is resistant to stabs, knife slashes, and high impacts. According to Gordon, this is a second attempt at building the suit after an earlier attempt five years ago. 
 ---------------------------------------------
predicted_summary : 

 A 21-year-old industrial design student, Jackson Gordon, creates an authentic, functional replica of Batman's suit. He raised funds for the project through Kickstarter and received $1,255 in six days. The armor is made with materials like Kevlar, memory foam, and ¼" Kydex plastic for strength, protection, and mobility. The cowl was more challenging to make, requiring a mold of Gordon's head, sculpting clay, and a two-part box mold. The final product has been approved by Legendary, and it's bee

In [6]:
#lets do eval 
import sys
sys.path.append("/home2/tathagato/summarization/MACSUM/experiments/")
from eval import output_length_metrics, output_topic_metrics

length_metrics = output_length_metrics(processed_zero_shot_length)
topic_metrics = output_topic_metrics(processed_zero_shot_topic)

[nltk_data] Downloading package punkt to /home2/tathagato/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home2/tathagato/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home2/tathagato/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


control_value short 121 121
Prediction Summary Length 129.9917355371901
Reference Summary Length 33.33884297520661
Prediction Compression Ratio 0.2130685383012572
Reference Compression Ratio 0.045536881494968894
control_value normal 298 298
Prediction Summary Length 174.6275167785235
Reference Summary Length 45.104026845637584
Prediction Compression Ratio 0.2510390804333993
Reference Compression Ratio 0.06021797495584893
control_value long 135 135
Prediction Summary Length 227.25925925925927
Reference Summary Length 90.66666666666667
Prediction Compression Ratio 0.38468564925088167
Reference Compression Ratio 0.12471463538151979
------------------------------
control value  short
CER 4.095151870931806
control value  normal
CER 4.431821511869199
control value  long
CER 2.4411110743742475
Overall
CER 3.8731880540799515
Relative Score:  0.1999591503267974
Absolute Prediction Score:  0.9305555555555554
Absolute Gold Score:  0.9761029411764706


## ZERO SHOT MISTRAL

In [9]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(length_metrics)

{   'long': {   'cer': 2.4411110743742475,
                'num_examples': 135,
                'prediction_compression_ratio': 0.38468564925088167,
                'prediction_summary_length': 227.25925925925927,
                'reference_compression_ratio': 0.12471463538151979,
                'reference_summary_length': 90.66666666666667},
    'normal': {   'cer': 4.431821511869199,
                  'num_examples': 298,
                  'prediction_compression_ratio': 0.2510390804333993,
                  'prediction_summary_length': 174.6275167785235,
                  'reference_compression_ratio': 0.06021797495584893,
                  'reference_summary_length': 45.104026845637584},
    'overall_cer': 3.8731880540799515,
    'short': {   'cer': 4.095151870931806,
                 'num_examples': 121,
                 'prediction_compression_ratio': 0.2130685383012572,
                 'prediction_summary_length': 129.9917355371901,
                 'reference_compression_rati

In [11]:
pp.pprint(topic_metrics)

{   'abs_gold_score': 0.9761029411764706,
    'abs_score': 0.9305555555555554,
    'relative_score': 0.1999591503267974}


## FINETUNED MISTRAL

In [12]:
processed_finetune_length = cleanup_output_dict(mistral_finetune_length, "length")
processed_finetune_topic = cleanup_output_dict(mistral_finetune_topic, "topic")

finetune_length_metrics = output_length_metrics(processed_finetune_length)
finetune_topic_metrics = output_topic_metrics(processed_finetune_topic)

control_value short 121 121
Prediction Summary Length 34.611570247933884
Reference Summary Length 33.33884297520661
Prediction Compression Ratio 0.05494300513780985
Reference Compression Ratio 0.045536881494968894
control_value normal 298 298
Prediction Summary Length 45.81543624161074
Reference Summary Length 45.104026845637584
Prediction Compression Ratio 0.06182018842540764
Reference Compression Ratio 0.06021797495584893
control_value long 135 135
Prediction Summary Length 58.67407407407408
Reference Summary Length 90.66666666666667
Prediction Compression Ratio 0.09195620445127325
Reference Compression Ratio 0.12471463538151979
------------------------------
control value  short
CER 0.66028238323305
control value  normal
CER 0.8809228996002723
control value  long
CER 0.557751904144753
Overall
CER 0.7539814070606894
Relative Score:  0.6149918300653593
Absolute Prediction Score:  0.4965277777777775
Absolute Gold Score:  0.9761029411764706


In [14]:
pp.pprint(finetune_length_metrics)
pp.pprint(finetune_topic_metrics)

{   'long': {   'cer': 0.557751904144753,
                'num_examples': 135,
                'prediction_compression_ratio': 0.09195620445127325,
                'prediction_summary_length': 58.67407407407408,
                'reference_compression_ratio': 0.12471463538151979,
                'reference_summary_length': 90.66666666666667},
    'normal': {   'cer': 0.8809228996002723,
                  'num_examples': 298,
                  'prediction_compression_ratio': 0.06182018842540764,
                  'prediction_summary_length': 45.81543624161074,
                  'reference_compression_ratio': 0.06021797495584893,
                  'reference_summary_length': 45.104026845637584},
    'overall_cer': 0.7539814070606894,
    'short': {   'cer': 0.66028238323305,
                 'num_examples': 121,
                 'prediction_compression_ratio': 0.05494300513780985,
                 'prediction_summary_length': 34.611570247933884,
                 'reference_compression_rat

In [24]:
def count_words(string):
    # Remove leading and trailing whitespace
    string = string.strip()
    
    # If the string is empty, return 0
    if not string:
        return 0
    
    # Split the string into words and count them
    words = string.split()
    return len(words)

# Example usage:
# text = "This is a sample sentence."
# word_count = count_words(text)
# print(f"Word count: {word_count}")

summaries = []

for key in processed_zero_shot_length.keys():
    summaries.append(processed_zero_shot_length[key]["predicted_summary"])
    print(processed_zero_shot_length[key]["predicted_summary"])
    print(count_words(processed_zero_shot_length[key]["predicted_summary"]))
    print("-------------------------------------------------")
print(len(summaries))

def extract_info_and_output(full_string, input_string):
    # Find the position of the input_string in the full_string
    input_index = full_string.find(input_string)
    
    # If the input_string is not found, return an error message
    if input_index == -1:
        print("seems ")
    
    # Extract some_basic_info (everything before input_string)
    some_basic_info = full_string[:input_index].strip()

    # Extract output (everything after input_string)
    output = full_string[input_index + len(input_string):].strip()

    return some_basic_info, output

Write a summary of the source text. The summary should be short in length. The length is defined in terms of number of words used in the summary. The source text is given below.  (CNN)Jackson Gordon is no ordinary 21-year-old. By day he is an industrial design student at Philadelphia University, but Gordon has another side to him -- a side altogether darker, tougher and more enigmatic. Hanging in his workshop Gordon has a full suit of armor plating, cape and cowl -- matte black and built to stop a knife. Gordon has an alter ego: the Dark Knight himself, Batman. You might expect his origin story to be cloaked in mystery, but speaking to CNN Gordon is quick to explain how the transformation took place. Gordon says his calling came five years ago when he began experimenting with cosplay. "Previously I'd been involved with costume making... I'd made a version of the Batsuit from Christopher Nolan's 'Dark Knight Trilogy' and I really liked that suit," Gordon says. But, as elaborate as his d

In [20]:
summaries = []
average = 0
for key in mistral_zero_shot_length.keys():
    summaries.append(mistral_zero_shot_length[key]["summary"])
    print(mistral_zero_shot_length[key]["summary"])
    print(count_words(mistral_zero_shot_length[key]["summary"]))
    average += count_words(mistral_zero_shot_length[key]["summary"])
    print("-------------------------------------------------")
print(average/len(mistral_zero_shot_length.keys()))

A 21-year-old industrial design student, Jackson Gordon, creates an authentic, functional replica of Batman's suit. He raised funds for the project through Kickstarter and received $1,255 in six days. The armor is made with materials like Kevlar, memory foam, and ¼" Kydex plastic for strength, protection, and mobility. The cowl was more challenging to make, requiring a mold of Gordon's head, sculpting clay, and a two-part box mold. The final product has been approved by Legendary, and it's been showcased at conventions and universities. Gordon's business, Armatus Design, plans to sell the cowls to the public in a few weeks, with demand coming from various communities such as martial artists, motorcycle riders, and even cosplay enthusiasts. The suit will not have the iconic bat symbol due to copyright issues. After completing his studies, Gordon intends to scale up the business, maintaining it as a one-man operation for now.
148
-------------------------------------------------
Jackson 